In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install PyMuPDF chromadb sentence-transformers tqdm regex
!pip install pdfplumber
!pip install fuzzywuzzy python-Levenshtein unidecode

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 119.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.4/20.4 MB 122.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 25.3 MB/s 

In [ ]:
import fitz
import re
import unicodedata
from tqdm import tqdm

PDF_PATH = "so_tay_2.pdf"
OUTPUT_TXT = "so_tay_clean.txt"
CHUNKS_TXT = "chunks.txt"

# ========== 1️⃣ Làm sạch cơ bản ==========
def normalize_text(text: str) -> str:
    text = unicodedata.normalize("NFKC", text)
    text = text.replace("–", "-")
    text = re.sub(r"(?<=\d),(?=\d)", ".", text)
    text = re.sub(r"[^\n a-zA-ZÀ-Ỹà-ỹ0-9.,:;()–\-+•\[\]]", " ", text)
    text = re.sub(r"[ \t]+", " ", text)
    text = re.sub(r"\s+([,.:;)\]\}])", r"\1", text)
    return text.strip()

# ========== 2️⃣ Ghép bảng thành dòng ngang ==========
def fix_grade_blocks(text: str) -> str:
    """
    Ghép các dòng dạng thang điểm dọc thành 1 dòng ngang liền mạch.
    """
    # Ghép các pattern kiểu “8.5 - 10 A” hoặc “B + 3.5”
    lines = text.splitlines()
    fixed_lines = []
    buffer = ""
    for line in lines:
        line = line.strip()
        if not line:
            if buffer:
                fixed_lines.append(buffer.strip())
                buffer = ""
            continue

        # Nối các dòng có vẻ là bảng điểm
        if re.match(r"^[0-9.,\-\+\sa-zA-Z]+$", line):
            buffer += " " + line
        else:
            if buffer:
                fixed_lines.append(buffer.strip())
                buffer = ""
            fixed_lines.append(line)

    if buffer:
        fixed_lines.append(buffer.strip())

    # Làm gọn lại các dòng bảng
    result = []
    for l in fixed_lines:
        # Ví dụ: "8.5 - 10 A 7.8 - 8.4 B + 7.0 - 7.7 B" → tách bằng regex và nối bằng "; "
        if re.search(r"\d+\.\d+\s*-\s*\d+\.\d+", l):
            parts = re.findall(r"\d+(?:\.\d+)?\s*-\s*\d+(?:\.\d+)?\s*[A-Z\+]*", l)
            if parts:
                result.append("; ".join(p.strip() for p in parts))
            else:
                result.append(l)
        else:
            result.append(l)
    return "\n".join(result)

# ========== 3️⃣ Tách điều / mục ==========
def split_sections(text):
    text = re.sub(r"Điều\s*\n\s*(\d+\.)", r"Điều \1", text, flags=re.IGNORECASE)
    text = re.sub(r"Mục\s*\n\s*(\d+\.)", r"Mục \1", text, flags=re.IGNORECASE)
    sections = re.split(r"(?=(?:\n?Điều\s+\d+\.|\n?Mục\s+\d+\.))", text, flags=re.IGNORECASE)
    sections = [s.strip() for s in sections if len(s.strip()) > 150]
    return sections

# ========== MAIN ==========
if __name__ == "__main__":
    print("🚀 Đang đọc PDF & ghép bảng điểm ngang...")
    doc = fitz.open(PDF_PATH)
    pages = []
    for page in tqdm(doc, desc="📖 Trích xuất"):
        text = page.get_text("text")
        text = normalize_text(text)
        text = fix_grade_blocks(text)
        pages.append(text)
    doc.close()

    full_text = "\n\n".join(pages)
    full_text = re.sub(r"SỔ TAY SINH VIÊN KHÓA\s*\d+", "", full_text)
    full_text = re.sub(r"\[\s*Trang\s*\d+\s*\]", "", full_text)
    full_text = re.sub(r"\n{3,}", "\n\n", full_text)

    with open(OUTPUT_TXT, "w", encoding="utf-8") as f:
        f.write(full_text)
    print(f"✅ Lưu văn bản sạch: {OUTPUT_TXT}")

    chunks = split_sections(full_text)
    with open(CHUNKS_TXT, "w", encoding="utf-8") as f:
        f.write("\n\n".join(chunks))
    print(f"✅ Tổng {len(chunks)} đoạn đã lưu vào {CHUNKS_TXT}")


# Tạo lại Collection khi chuyển tài khoản

In [ ]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import os
import chromadb.api

# === 1️⃣ Cấu hình ===
DB_DIR = "/content/drive/MyDrive/chatbotHCMUE/content/vector_store" # Vị trí Collection mới sẽ được lưu
COLLECTION = "so_tay_hcmue"
DRIVE_MODEL_PATH = '/content/drive/MyDrive/chatbotHCMUE/bge-m3-model/'
INPUT_TXT = "/content/drive/MyDrive/chatbotHCMUE/content/chunks.txt" # Tệp nguồn

# === 2️⃣ Đọc Dữ liệu Nguồn ===
try:
    with open(INPUT_TXT, "r", encoding="utf-8") as f:
        # Tách nội dung theo hai dòng trống và lọc các đoạn ngắn
        docs = [x.strip() for x in f.read().split("\n\n") if len(x.strip()) > 80]
    print(f"📘 Tổng {len(docs)} đoạn nội dung đã được tải từ {INPUT_TXT}.")
except FileNotFoundError:
    print(f"❌ Lỗi: Không tìm thấy file nguồn '{INPUT_TXT}'.")
    docs = []

if not docs:
    print("⚠️ Không có dữ liệu để tạo Collection. Vui lòng kiểm tra lại file nguồn.")
    # Kết thúc chương trình nếu không có docs
else:
    # === 3️⃣ RESET và TẠO LẠI Collection ===

    # Khởi tạo client
    chromadb.api.client.SharedSystemClient._instance = None
    # Settings(allow_reset=True) giúp dễ dàng tạo lại
    client = chromadb.PersistentClient(path=DB_DIR, settings=Settings(allow_reset=True))

    # Xóa Collection cũ (nếu có, để đảm bảo tạo mới)
    try:
        client.delete_collection(COLLECTION)
    except:
        pass # Bỏ qua lỗi nếu Collection chưa tồn tại

    # Tạo Collection mới
    col = client.create_collection(COLLECTION)
    print(f"✅ Đã tạo mới Collection '{COLLECTION}'.")

    # === 4️⃣ Tạo Embeddings và Thêm vào Collection ===
    try:
        model = SentenceTransformer(DRIVE_MODEL_PATH)
        print("🚀 Đang tạo embeddings...")

        embs = []
        # Xử lý theo từng lô (batch) để tiết kiệm bộ nhớ và thời gian
        for i in tqdm(range(0, len(docs), 32)):
            batch = docs[i:i+32]
            # Tạo embeddings cho từng lô
            batch_emb = model.encode(batch, normalize_embeddings=True).tolist()
            embs.extend(batch_emb)

        # Thêm dữ liệu vào Collection
        col.add(
            ids=[str(i) for i in range(len(docs))],
            documents=docs,
            embeddings=embs
        )

        final_count = col.count()
        print(f"\n✅ Đã lưu {final_count} đoạn vào collection '{COLLECTION}'.")
        print("Collection đã được tạo lại thành công và sẵn sàng để sử dụng.")

    except Exception as e:
        print(f"\n❌ Lỗi trong quá trình tạo embeddings/add dữ liệu: {e}")

📘 Tổng 411 đoạn nội dung đã được tải từ /content/drive/MyDrive/chatbotHCMUE/content/chunks.txt.
✅ Đã tạo mới Collection 'so_tay_hcmue'.
🚀 Đang tạo embeddings...


100%|██████████| 13/13 [00:35<00:00,  2.76s/it]



✅ Đã lưu 411 đoạn vào collection 'so_tay_hcmue'.
Collection đã được tạo lại thành công và sẵn sàng để sử dụng.


# Tải lại Collection

In [ ]:
import chromadb
from chromadb.config import Settings
import os

DB_DIR = "/content/drive/MyDrive/chatbotHCMUE/content/vector_store"
COLLECTION = "so_tay_hcmue"

if not os.path.isdir("/content/drive"):
    print("⚠️ Lỗi: Google Drive chưa được Mount.")
elif not os.path.isdir(DB_DIR):
    print(f"⚠️ Lỗi: Không tìm thấy thư mục DB tại: {DB_DIR}")
else:
    # === 2️⃣ Tải lại Collection ===
    try:
        # 1. Khởi tạo client, nó sẽ tự động load dữ liệu từ DB_DIR
        client = chromadb.PersistentClient(path=DB_DIR, settings=Settings())

        # 2. KIỂM TRA và Khắc phục lỗi "count 0"
        # Thử get_or_create: Nếu collection đã tồn tại, nó sẽ lấy về;
        # Nếu không, nó sẽ tạo một cái mới (nhưng trong trường hợp của bạn, nó sẽ đọc từ các file đã có)
        col = client.get_or_create_collection(COLLECTION)

        # 3. Kiểm tra lại:
        current_count = col.count()
        print(f"\n✅ Đã tải thành công Collection '{COLLECTION}' từ Drive.")
        print(f"Tổng số tài liệu trong Collection: {current_count}")

        if current_count == 0:
            print("⚠️ LỖI VẪN TIẾP DIỄN: Dữ liệu DB có thể bị hỏng hoặc lỗi quyền truy cập.")
            print("Vui lòng kiểm tra lại cấu trúc thư mục 'vector_store' trong Drive mới.")

    except Exception as e:
        print(f"\n❌ Lỗi: {e}")


✅ Đã tải thành công Collection 'so_tay_hcmue' từ Drive.
Tổng số tài liệu trong Collection: 0
⚠️ LỖI VẪN TIẾP DIỄN: Dữ liệu DB có thể bị hỏng hoặc lỗi quyền truy cập.
Vui lòng kiểm tra lại cấu trúc thư mục 'vector_store' trong Drive mới.


# Kiểm tra Collection

In [ ]:
import chromadb
from chromadb.config import Settings
DB_DIR = "/content/drive/MyDrive/chatbotHCMUE/content/vector_store"
COLLECTION = "so_tay_hcmue"

try:
    client = chromadb.PersistentClient(path=DB_DIR, settings=Settings())
    col = client.get_collection(COLLECTION)
    count = col.count()
    print(f"Tổng số tài liệu hiện tại trong Collection: {count}")

    if count == 0:
        print("❌ LỖI: Collection bị rỗng. Bạn phải chạy lại code tạo Collection.")
    elif count < 400: # Ví dụ: nếu số lượng gốc của bạn là 409
        print("⚠️ LỖI: Dữ liệu bị thiếu. Bạn phải chạy lại code tạo Collection.")
    else:
        print("✅ Dữ liệu đầy đủ.")

except Exception as e:
    print(f"❌ LỖI NẶNG: Không thể tải Collection. {e}")

Tổng số tài liệu hiện tại trong Collection: 411
✅ Dữ liệu đầy đủ.


# Tải model

In [ ]:
from sentence_transformers import SentenceTransformer
import os

# Đường dẫn nơi bạn muốn lưu mô hình trên Google Drive
DRIVE_MODEL_PATH = '/content/drive/MyDrive/chatbotHCMUE/bge-m3-model/'

# Tạo thư mục nếu nó chưa tồn tại
os.makedirs(DRIVE_MODEL_PATH, exist_ok=True)
print(f"Thư mục lưu mô hình: {DRIVE_MODEL_PATH}")

Thư mục lưu mô hình: /content/drive/MyDrive/chatbotHCMUE/bge-m3-model/


In [ ]:
from sentence_transformers import SentenceTransformer
EMB_MODEL_NAME = "BAAI/bge-m3"

# Khởi tạo mô hình (đã tải hoặc đang tải nếu chưa có trong cache)
model = SentenceTransformer(EMB_MODEL_NAME)

# Lưu mô hình vào Drive
# Đây là bước quan trọng nhất
model.save(DRIVE_MODEL_PATH)

print("✅ Đã lưu mô hình thành công vào Google Drive.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

✅ Đã lưu mô hình thành công vào Google Drive.


# Truy vấn dữ liệu

In [ ]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import re, json, pandas as pd
from fuzzywuzzy import fuzz
from unidecode import unidecode
import chromadb.api

# === 0️⃣ Hàm Loại bỏ Dấu Tiếng Việt (Hỗ trợ Fuzzy Matching) ===
def remove_vietnamese_diacritics(text):
    """Sử dụng unidecode để loại bỏ dấu tiếng Việt và chuyển sang chữ thường."""
    return unidecode(text).lower()


# === 1️⃣ Cấu hình ===
DB_DIR = "/content/drive/MyDrive/chatbotHCMUE/content/vector_store"
DRIVE_MODEL_PATH = '/content/drive/MyDrive/chatbotHCMUE/bge-m3-model/'
COLLECTION = "so_tay_hcmue"
TABLE_JSON = "/content/drive/MyDrive/chatbotHCMUE/so_tay_all_tables_clean.json"


# === 3️⃣ Hàm in đẹp text ===
def pretty(text: str) -> str:
    text = re.sub(r"([;:.\)\]\}])\s*(?=[\+\-•–])", r"\1\n", text)
    text = re.sub(r"\s*([+\-•–])\s+", r"\n\1 ", text)
    text = re.sub(r"\s*(Điều\s+\d+\.)", r"\n\1", text, flags=re.IGNORECASE)
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


# === 4️⃣ Hàm tra bảng JSON (Cải tiến Khớp Mờ Chính Xác) ===
def find_table_by_keyword(query: str):

    normalized_query = remove_vietnamese_diacritics(query)

    mapping = {
        # Thêm các key đơn giản để tăng khả năng khớp chính xác
        "4 sang chữ": ["thang_diem_4"],
        "học bổng": ["xep_loai_hoc_bong"],
        "chữ sang 10": ["thang_diem_10_chu"],
        "thang điểm 10 sang chữ": ["thang_diem_10_chu"],
        "thang điểm 4 sang chữ": ["thang_diem_4"],
        "xếp loại học lực": ["xep_loai_hoc_luc"],
        "xếp loại học bổng": ["xep_loai_hoc_bong"],
        "yêu cầu học bổng": ["yeu_cau_hoc_bong"],
        "điểm rèn luyện": ["diem_ren_luyen1", "diem_ren_luyen2", "diem_ren_luyen3", "diem_ren_luyen4", "diem_ren_luyen5"],
    }

    BEST_MATCH_SCORE = 90 # Tăng ngưỡng tin cậy lên 90 để tránh khớp nhầm
    best_match_key = None
    best_score = 0
    final_result = ""

    # 1. KIỂM TRA KHỚP HOÀN HẢO (CHÍNH XÁC) TRƯỚC
    for map_key in mapping.keys():
        if normalized_query == remove_vietnamese_diacritics(map_key):
            best_match_key = map_key
            best_score = 100
            break

    # 2. KHỚP MỜ (FUZZY MATCHING)
    if not best_match_key:
        for map_key, types in mapping.items():
            normalized_map_key = remove_vietnamese_diacritics(map_key)

            # Sử dụng WRatio và Partial Ratio
            score_wratio = fuzz.WRatio(normalized_query, normalized_map_key)
            score_partial = fuzz.partial_ratio(normalized_query, normalized_map_key)

            # Ưu tiên điểm cao nhất từ hai phương pháp
            score = max(score_wratio, score_partial)

            if score > best_score and score >= BEST_MATCH_SCORE:
                best_score = score
                best_match_key = map_key

    # 3. Xử lý và THU THẬP TẤT CẢ các bảng liên quan
    if best_match_key:
        types_to_export = mapping[best_match_key]

        final_result += f"### 📊 Kết quả Tra cứu Bảng (Độ khớp: {best_score}%)\n"

        for type_name in types_to_export:
            for t in tables:
                if t["type"] == type_name:
                    df = pd.DataFrame(t["data"])

                    table_title = t.get("title", type_name.replace("_", " ").title())

                    final_result += f"\n#### Bảng: {table_title}\n"
                    final_result += df.to_markdown(index=False)
                    final_result += "\n"

    # 4. Trả về kết quả tổng hợp
    if final_result:
        return final_result
    else:
        return None


# === 5️⃣ Giao diện chat CLI ===
print("=== 🔍 Tra cứu Sổ tay Sinh viên HCMUE (có cả bảng) ===")
print("Nhập 'exit' để thoát.\n")

# === 2️⃣ Nạp dữ liệu ===
# Reset instance tránh lỗi "already exists"
chromadb.api.client.SharedSystemClient._instance = None
try:
    client = chromadb.PersistentClient(path=DB_DIR, settings=Settings())
    col = client.get_collection(COLLECTION)
    model = SentenceTransformer(DRIVE_MODEL_PATH)
    print("✅ Đã tải mô hình và Collection thành công.")
except Exception as e:
    print(f"❌ Lỗi khởi tạo hệ thống: {e}. Vui lòng kiểm tra lại trạng thái Drive/Mô hình/Collection.")
    client, col, model = None, None, None


try:
    with open(TABLE_JSON, "r", encoding="utf-8") as f:
        tables = json.load(f)
except FileNotFoundError:
    print("⚠️ Không tìm thấy file bảng JSON. Chỉ tra cứu văn bản được.")
    tables = []


while True:
    q = input("📘 Nhập nội dung muốn tra cứu: ").strip()
    if not q:
        continue
    if q.lower() == "exit":
        print("👋 Tạm biệt!")
        break

    # Kiểm tra nếu hệ thống không được khởi tạo
    if not all([client, col, model]):
        print("❌ HỆ THỐNG CHƯA SẴN SÀNG. Vui lòng kiểm tra lại lỗi khởi tạo ở trên.")
        print("\n-----------------------------\n")
        continue

    # KIỂM TRA LỖI 2: Yêu cầu nhập thêm khi chuỗi quá ngắn
    if len(remove_vietnamese_diacritics(q)) < 5:
        print("⚠️ Vui lòng nhập chi tiết hơn (ví dụ: 'điểm rèn luyện', 'học bổng').")
        print("\n-----------------------------\n")
        continue

    # --- Kiểm tra có bảng phù hợp không ---
    table_result = find_table_by_keyword(q)
    if table_result:
        print("\n🔎 Kết quả tìm thấy (BẢNG):\n")
        print(table_result)
        print("\n-----------------------------\n")
        continue

    # --- Nếu không có bảng, tra văn bản như cũ ---
    try:
        q_emb = model.encode(q, normalize_embeddings=True).tolist()
        res = col.query(query_embeddings=[q_emb], n_results=3)
    except Exception as e:
        print(f"\n❌ Lỗi Encoding/Truy vấn: {e}. Vui lòng kiểm tra lại trạng thái mô hình/Collection.")
        print("\n-----------------------------\n")
        continue

    # KIỂM TRA KẾT QUẢ TRUY VẤN
    if not res["documents"] or not res["documents"][0] or not res["documents"][0][0].strip():
        print("\n❌ Không tìm thấy thông tin liên quan trong Sổ tay Sinh viên.")
        print("Vui lòng thử từ khóa khác hoặc chi tiết hơn.")
        print("\n-----------------------------\n")
        continue

    # --- Nếu có kết quả, in ra như thường ---
    print("\n🔎 Kết quả tìm thấy (VĂN BẢN):")
    for i, doc in enumerate(res["documents"][0]):
        # Chỉ in ra đoạn không trống
        if doc.strip():
            print(f"\n📄 Đoạn {i+1}:")
            print(pretty(doc))
    print("\n-----------------------------\n")

=== 🔍 Tra cứu Sổ tay Sinh viên HCMUE (có cả bảng) ===
Nhập 'exit' để thoát.

✅ Đã tải mô hình và Collection thành công.
📘 Nhập nội dung muốn tra cứu: aaaaaaaaaaaaaaaaa

🔎 Kết quả tìm thấy (VĂN BẢN):

📄 Đoạn 1:
212 
19. Khoa Giáo dục Tiểu học
Điện thoại liên lạc: (028) 38352020
số máy nội bộ: 134, 135
Email: khoagdth hcmue.edu.vn
Website: khoagdth.hcmue.edu.vn
Văn phòng làm việc: Nhà A, tầng 3, P.308, P.309, P.310,
số 280 An Dương Vương,
phường Chợ Quán, TP. HCM.
20. Khoa Giáo dục Mầm non
Điện thoại liên lạc: (028) 38352020
số máy nội bộ: 133
Email: khoagdmn hcmue.edu.vn
Website: khoagdmn.hcmue.edu.vn
Văn phòng làm việc: Nhà A, tầng 3, P.306, P.307,
số 280 An Dương Vương,
phường Chợ Quán, TP. HCM.
21. Khoa Giáo dục Quốc phòng
Điện thoại liên lạc: (028) 38352020
số máy nội bộ: 165, 167
Email: khoagdqp hcmue.edu.vn
Website: khoagdqp.hcmue.edu.vn
Văn phòng làm việc: Nhà C, tầng 1, P.106, P.108,
số 280 An Dương Vương,
phường Chợ Quán, TP. HCM.
22. Khoa Giáo dục Thể chất
Điện thoại liên lạc: